Following [this tutorial](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# N-Grams

In [2]:
word_to_ix = {"hello": 0, "world": 1}

embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings

lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])


[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [4]:

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[520.9389450550079, 518.4947490692139, 516.0660655498505, 513.6526546478271, 511.25297570228577, 508.86681938171387, 506.494104385376, 504.13247895240784, 501.7818763256073, 499.4411427974701]
tensor([-0.9735,  0.1031,  0.4429, -0.5230,  0.4934, -0.5362, -0.6959,  0.5455,
        -1.0910, -0.2009], grad_fn=<SelectBackward0>)


# CBOW

In [39]:
CONTEXT_SIZE = 5  # 2 words to the left, 2 to the right
# raw_text = """We are about to study the idea of a computational process.
# Computational processes are abstract beings that inhabit computers.
# As they evolve, processes manipulate other abstract things called data.
# The evolution of a process is directed by a pattern of rules
# called a program. People create programs to direct processes. In effect,
# we conjure the spirits of the computer with our spells.""".split()

import pandas as pd

raw_df = pd.read_csv("../sentiment_analysis/data/imdb_data.csv")

In [40]:
raw_text = " ".join(raw_df["review"]).split()

In [41]:
vocab = set(raw_text)
vocab_list = list(vocab)
vocab_size = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab_list)}

data = []

for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = raw_text[i - CONTEXT_SIZE : i] + raw_text[i + 1 : i + CONTEXT_SIZE + 1]
    target = raw_text[i]

    data.append((context, target))


def make_context_vector(context, word_to_ix) -> torch.Tensor:
    output = torch.zeros(len(word_to_ix), dtype=torch.float)
    for w in context:
        output[word_to_ix[w]] += 1

    return output / len(context)


def make_target_vector(target: str, word_to_ix) -> torch.Tensor:
    return torch.tensor([word_to_ix[target]], dtype=torch.long)

In [42]:
class CBOW(nn.Module):

    def __init__(self, vocab_size: int, embedding_size: int):
        super().__init__()
        self.w_n = nn.Linear(vocab_size, embedding_size, bias=False)
        self.w_v = nn.Linear(embedding_size, vocab_size, bias=False)

    def forward(self, inputs: torch.Tensor):
        return self.w_v(self.w_n(inputs))

In [43]:
from tqdm import tqdm

n_epochs = 10
embedding_size = 20
learning_rate = 0.1

model = CBOW(vocab_size=vocab_size, embedding_size=embedding_size)

loss_function = F.cross_entropy
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in tqdm(range(n_epochs)):
    for context, target in data:
        input = make_context_vector(context, word_to_ix=word_to_ix)
        label = make_target_vector(target, word_to_ix=word_to_ix)

        optimizer.zero_grad()

        output = model(input)
        loss = loss_function(output, label[0])
        loss.backward()

        optimizer.step()

  0%|          | 0/10 [00:00<?, ?it/s]

In [35]:
# interactive t-sne plot of the embeddings using plotly
from sklearn.manifold import TSNE
import plotly.graph_objects as go

tsne = TSNE(n_components=2, random_state=0, perplexity=10)
all_word_vectors_matrix = model.w_n.weight.data.numpy().T
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

points = go.Scatter(
    x=all_word_vectors_matrix_2d[:, 0],
    y=all_word_vectors_matrix_2d[:, 1],
    mode='markers+text',
    text=vocab_list,
    textposition='top center',
    marker=dict(
        size=10,
        color='rgba(152, 0, 0, .8)',
        line=dict(
            width=2,
            color='rgb(0, 0, 0)'
        )
    )
)

fig = go.Figure(data=[points])
fig.show()


In [10]:
import sys
sys.executable

'/home/jerome/source/pytorch_hello_worlds/venv/bin/python'

In [14]:
all_word_vectors_matrix.size

980

In [21]:
all_word_vectors_matrix.shape

(20, 49)

In [23]:
vocab

{'As',
 'Computational',
 'In',
 'People',
 'The',
 'We',
 'a',
 'about',
 'abstract',
 'are',
 'beings',
 'by',
 'called',
 'computational',
 'computer',
 'computers.',
 'conjure',
 'create',
 'data.',
 'direct',
 'directed',
 'effect,',
 'evolution',
 'evolve,',
 'idea',
 'inhabit',
 'is',
 'manipulate',
 'of',
 'other',
 'our',
 'pattern',
 'process',
 'process.',
 'processes',
 'processes.',
 'program.',
 'programs',
 'rules',
 'spells.',
 'spirits',
 'study',
 'that',
 'the',
 'they',
 'things',
 'to',
 'we',
 'with'}